In [ ]:
import requests
import json
import os
from datetime import datetime

In [ ]:



congress_url = "https://dadosabertos.camara.leg.br/api/v2/"
current_year = datetime.now().year
years = list(range(current_year, 2007, -1))



def get_all_congress_members_year(year):
    print(f'------------{year}------------')
    if not os.path.exists(f'data/deputados-{year}.json'):
        all_members_response = requests.get(f'{congress_url}/deputados')

        if(all_members_response.status_code == 200):
            with open(f'data/deputados-{year}.json', 'w', encoding='utf-8') as file:
                json.dump(all_members_response.json(), file, ensure_ascii=False)
    else:
        print(f'File for year {year} already exists. Exiting')



In [74]:
for year in years:
    get_all_congress_members_year(year)

------------2025------------
File for year 2025 already exists. Exiting
------------2024------------
File for year 2024 already exists. Exiting
------------2023------------
File for year 2023 already exists. Exiting
------------2022------------
File for year 2022 already exists. Exiting
------------2021------------
File for year 2021 already exists. Exiting
------------2020------------
File for year 2020 already exists. Exiting
------------2019------------
File for year 2019 already exists. Exiting
------------2018------------
File for year 2018 already exists. Exiting
------------2017------------
File for year 2017 already exists. Exiting
------------2016------------
File for year 2016 already exists. Exiting
------------2015------------
File for year 2015 already exists. Exiting
------------2014------------
File for year 2014 already exists. Exiting
------------2013------------
File for year 2013 already exists. Exiting
------------2012------------
File for year 2012 already exists. 

In [ ]:
political_parties_2008 = {}


with open('data/deputados-2008.json', 'r', encoding='utf-8') as file:
    data_2008 = json.load(file)
    
for i in len(data_2008):
    political_parties_2008.append()

512


In [ ]:
dict = {}

data = ['pt', 'pt', 'pt', 'pl', 'pl', 'pmdb']

for d in data:
    if d not in dict:
        dict[d] = 1
    else:
        dict[d] = dict[d]+1

print(dict)


TypeError: 'int' object is not iterable